# Initialization

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import time

In [2]:
# Import probability matrices

P_10 = np.loadtxt('HW2_2022402309_10.txt')
P_100 = np.loadtxt('HW2_2022402309_100.txt')
P_1000 = np.loadtxt('HW2_2022402309_1000.txt')


# Task 1

## Method 1

In [3]:
# We will implement the formula (I - P.T)π = 0 and sum(π_j = 1)

def analytical(P):
    # Shape of the transition matrix
    dim = P.shape[0]
    
    # Create the coefficient matrix A = (I - P.T)
    A = np.eye(dim) - P.T
    
    # Append an equation to the linear system so the probabilities sum to 1
    A = np.vstack((A, np.ones(dim)))
    
    # Create the right-hand side vector
    b = np.zeros(dim)
    b = np.append(b, 1)
    
    # Solve the linear system
    pi = np.linalg.lstsq(A, b, rcond=None)[0]
    
    return pi

## Method 2

In [4]:
def simulation(T, P):
    # Number of states
    num_states = len(P)
    
    # Compute cumulative probabilities for each state
    cumulative_probs = np.cumsum(P, axis=1)
    
    # Choose a random initial state
    new_state = np.random.randint(0, num_states)
    
    # Initialize the occurrence times of the states
    realization_count = np.zeros(num_states, dtype=int)
    
    for i in range(T):
        # Generate a random number
        RN = np.random.rand()
        
        # Find the next state using binary search
        new_state = np.searchsorted(cumulative_probs[new_state], RN)
        
        # Increment the realization times
        realization_count[new_state] += 1

    # Normalize the realization counts
    steady_state_prob = realization_count / T
    
    return steady_state_prob

## Method 3

In [5]:
def matrix_multiplication(P, eps):

    # Initialize the matrix to be used
    new_matrix = P
    # Initialize the epsilon matrix to be used
    eps_matrix = np.full_like(P, eps)
    # Get the dimension of the input matrix
    dim = P.shape[0]

    while 1:
        # Multiply P^2 with itself
        new_matrix = np.dot(new_matrix, new_matrix)
        # Evaluate pi j bar
        col_means = np.mean(new_matrix, axis=0)
        # Evaluate abs(P^2_t - pi j bar)
        abs_diff = np.abs(new_matrix - col_means)  # Calculate absolute difference

        # Check whether the abs_diff matrix has smaller entries than epsilon for all i, j
        count = 0
        for i in range(dim):
            for j in range(dim):
                if abs_diff[i,j] < eps_matrix[i,j]:
                    count += 1

        # If yes break out of while loop
        if count == dim**2:
            break
        
    return new_matrix[0]

# Task 2

In [6]:
# MSE function

def MSE(analytical, other):

    dim = len(analytical)
    sum = 0
    for i in range(dim):
        diff_sq = (analytical[i] - other[i])**2
        sum += diff_sq
    return sum

## Method 1

In [7]:
# For 10 x 10

start_time = time.time()
a_10 = analytical(P_10)
end_time = time.time()
runtime_a_10 = end_time - start_time
print("Runtime for 10x10:", runtime_a_10, "seconds")

# For 100 x 100

start_time = time.time()
a_100 = analytical(P_100)
end_time = time.time()
runtime_a_100 = end_time - start_time
print("Runtime for 100x100:", runtime_a_100, "seconds")

# For 1000 x 1000

start_time = time.time()
a_1000 = analytical(P_1000)
end_time = time.time()
runtime_a_1000 = end_time - start_time
print("Runtime for 1000x1000:", runtime_a_1000, "seconds")

Runtime for 10x10: 0.0001418590545654297 seconds
Runtime for 100x100: 0.0028781890869140625 seconds
Runtime for 1000x1000: 0.35988497734069824 seconds


## Method 2

In [8]:
# For 10 x 10

start_time = time.time()
s_10 = simulation(10**7, P_10)
end_time = time.time()
runtime_s_10 = end_time - start_time
print("Runtime for 10x10:", runtime_s_10, "seconds")

MSE_s_10 = MSE(a_10, s_10)
print("MSE for 10x10:", MSE_s_10)

# For 100 x 100

start_time = time.time()
s_100 = simulation(10**7, P_100)
end_time = time.time()
runtime_s_100 = end_time - start_time
print("Runtime for 100x100:", runtime_s_100, "seconds")

MSE_s_100 = MSE(a_100, s_100)
print("MSE for 100x100:", MSE_s_100)


# For 1000 x 1000

start_time = time.time()
s_1000 = simulation(10**7, P_1000)
end_time = time.time()
runtime_s_1000 = end_time - start_time
print("Runtime for 1000x1000:", runtime_s_1000, "seconds")

MSE_s_1000 = MSE(a_1000, s_1000)
print("MSE for 1000x1000:", MSE_s_1000)

Runtime for 10x10: 9.013199806213379 seconds
MSE for 10x10: 3.392188745087679e-08
Runtime for 100x100: 9.195012092590332 seconds
MSE for 100x100: 1.2165643854766073e-07
Runtime for 1000x1000: 9.747640132904053 seconds
MSE for 1000x1000: 1.0043015546933541e-07


## Method 3

In [9]:
# For 10 x 10

start_time = time.time()
m_10 = matrix_multiplication(P_10, 10**-6)
end_time = time.time()
runtime_m_10 = end_time - start_time
print("Runtime for 10x10:", runtime_m_10, "seconds")

MSE_m_10 = MSE(a_10, m_10)
print("MSE for 10x10:", MSE_m_10)

# For 100 x 100

start_time = time.time()
m_100 = matrix_multiplication(P_100, 10**-6)
end_time = time.time()
runtime_m_100 = end_time - start_time
print("Runtime for 100x100:", runtime_m_100, "seconds")

MSE_m_100 = MSE(a_100, m_100)
print("MSE for 100x100:", MSE_m_100)


# For 1000 x 1000

start_time = time.time()
m_1000 = matrix_multiplication(P_1000, 10**-6)
end_time = time.time()
runtime_m_1000 = end_time - start_time
print("Runtime for 1000x1000:", runtime_m_1000, "seconds")

MSE_m_1000 = MSE(a_1000, m_1000)
print("MSE for 1000x1000:", MSE_m_1000)

Runtime for 10x10: 0.0002009868621826172 seconds
MSE for 10x10: 4.853811034335566e-19
Runtime for 100x100: 0.030458927154541016 seconds
MSE for 100x100: 1.1212114559314543e-16
Runtime for 1000x1000: 0.42735910415649414 seconds
MSE for 1000x1000: 1.2437491913160045e-12


# Task 3

In [10]:
data = {
    "MSE Dataset 1": ["-", MSE_s_10, MSE_m_10],
    "Runtime Dataset 1 (s)": [runtime_a_10, runtime_s_10, runtime_m_10],
    "MSE Dataset 2": ["-", MSE_s_100, MSE_m_100],
    "Runtime Dataset 2 (s)": [runtime_a_100, runtime_s_100, runtime_m_100],
    "MSE Dataset 3": ["-", MSE_s_1000, MSE_m_1000],
    "Runtime Dataset 3 (s)": [runtime_a_1000, runtime_s_1000, runtime_m_1000],
}

index = ["Method 1", "Method 2", "Method 3"]

df = pd.DataFrame(data, index=index)
df

,MSE Dataset 1,Runtime Dataset 1 (s),MSE Dataset 2,Runtime Dataset 2 (s),MSE Dataset 3,Runtime Dataset 3 (s)
Method 1,-,0.000142,-,0.002878,-,0.359885
Method 2,0.0,9.013200,0.0,9.195012,0.0,9.747640
Method 3,0.0,0.000201,0.0,0.030459,0.0,0.427359


# Task 4

The results are consistent with expectations. Across all methods, the steady-state probabilities are very similar. Notably, Method 3 is more realiable compared to Method 2, evidenced by its lower Mean Squared Error (MSE), although Method 2 also demonstrates a very low MSE. Moreover, Method 3 achieves these results with significantly fewer iterations, leading to reduced computation time. The number of iterations varies across datasets in Method 3. Method 3, with its fewer iterations, can be considered more efficient than Method 2, which requires 10^7 iterations.

# Task 5

In [11]:
P_10_modified = P_10.copy()

In [12]:
for i in range(10):
    P_10_modified[0][i] = 0

P_10_modified[0][0] = 1

In [14]:
print(analytical(P_10_modified))
print(simulation(10**7,P_10_modified))
print(matrix_multiplication(P_10_modified, 10**-6))


[ 1.00000000e+00 -4.15658897e-16 -4.56298154e-16 -4.74341283e-16
 -3.42152703e-16 -4.84068440e-16 -4.77580090e-16 -4.17105800e-16
 -4.99482958e-16 -4.54782983e-16]
[9.999993e-01 3.000000e-07 0.000000e+00 1.000000e-07 1.000000e-07
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 2.000000e-07]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


As anticipated, the results align with our expectations. Given that the 0th state is now absorbing, this Markov Chain is no longer ergodic. Consequently, the long-term proportions of the 0th state will converge to 1, while all other states will tend towards 0 (i.e., pi_j = 0 for all states except state 0). Additionally, the 0th state in Method 2 does not directly equal to 1. As discussed earlier, this underscores the fact that Method 2 is the least reliable among the methods considered.